In [1]:
import numpy as np
import sys
from os.path import join as oj
sys.path.insert(1, oj(sys.path[0], 'train_model'))

from train_model import sent_util

import torch
from torchtext import data, datasets

In [2]:
# To train model, first run 'train.py' from train_model dir

# get model
snapshot_dir = 'train_model/results/'

snapshot_file = oj(snapshot_dir, 
                    'best_snapshot_devacc_84.9770642201835_devloss_0.5329645872116089_iter_7000_model.pt')
model = sent_util.get_model(snapshot_file)

# get data
inputs, answers, train_iterator, dev_iterator = sent_util.get_sst()

loading train_model/results/best_snapshot_devacc_84.9770642201835_devloss_0.5329645872116089_iter_7000_model.pt
loaded onto gpu...


In [3]:
# Find sentence used in figure 2
batch_nums = list(range(6920))
data = sent_util.get_batches(batch_nums, train_iterator, dev_iterator) 
for ind in range(6919):
    text = data[ind].text.data[:, 0]
    words = [inputs.vocab.itos[i] for i in text]
    if words[0] == 'it' and words[1] == "'s" and words[2] == 'easy':
        high_level_comp_ind = ind
        break

getting batches...


In [4]:
# Produce CD importance scores for phrases used in figure 2
pos, pos_irrel = sent_util.CD(data[high_level_comp_ind], model, start = 0, stop = 15)
print(' '.join(words[:16]), pos[0] - pos[1])
neg, neg_irrel = sent_util.CD(data[high_level_comp_ind], model, start = 16, stop = 26)
print(' '.join(words[16:]), neg[0] - neg[1])

it 's easy to love robin tunney -- she 's pretty and she can act -- 0.545045315382
but it gets harder and harder to understand her choices . -1.22609390466


In [5]:
# Sanity check: CD is a decomposition, so an effective way to check for bugs is to verify that the decomposition holds (up to numerical errors)
print(pos + pos_irrel)
linear_bias = model.hidden_to_label.bias.data.cpu().numpy()
print((model(data[high_level_comp_ind]).data.cpu().numpy() - linear_bias)[0])

[-0.07590632  0.28036043]


train_model/model.py:30: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  lstm_out, self.hidden = self.lstm(vecs, self.hidden)


[-0.07590645  0.28036064]
